![](./Ridge_Plot_First_Wave_vs_Second_Wave.jpg)

# # UK COVID-19 Analysis

Note: Data and graphs are updated as new data is available - however this means written analysis is out of date (last update: 4th Dec 2020). Although it is largely still applicable, there may be some interesting recent phenomena not discussed.
__________________________________________________

Hi all - this is a hotch-potch of analyses that I've been doing using the [UK COVID-19 Dataset](https://www.kaggle.com/vascodegama/uk-covid19-data), scraped from the official UK government COVID-19 reporting website.

I am from the UK, so I wanted to understand a bit more about the regional variations across the country, the effect of lockdown measures on different regions, and potentially bring in some other datasets to try to ascertain some of the key factors that have led to those regional differences. I am originally from the North-West of England, so forgive my bias in focusing on that region at times!

I've been working on it for a while and I don't really have an endpoint in mind, so I thought that I'd make it public anyway in case anybody finds it interesting or useful. I'm focused primarily on the daily deaths and cumulative death-rates (per 100,000 population) data as the variability in testing capability throughout the pandemic makes it difficult to get much out of case numbers.

I've also focused on the deaths and death-rate by *death date*, as opposed to *publication date* which can be misleading for tracking the prevalence of the virus over time - for an explanation of why this is, see this article (and many really good articles) from [Oxford Centre for Evidence-Based Medicene](https://www.cebm.net/covid-19/reconciling-covid-19-death-data-in-the-uk/)

# Imports
______________________________________________________________________________

In [ ]:
# General Imports
import numpy as np 
import pandas as pd      
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import date, timedelta
import scipy.stats

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load Data and Create DataFrames
______________________________________________________________________________

In [ ]:
# Load CSV data to pandas DataFrames

UK_TOTAL_DATA = pd.read_csv('../input/uk-covid19-data/UK_National_Total_COVID_Dataset.csv', index_col='date',parse_dates=True) 
DEVOLVED_NATION_DATA = pd.read_csv('../input/uk-covid19-data/UK_Devolved_Nations_COVID_Dataset.csv', index_col='date',parse_dates=True)
ENGLAND_REGIONS_DATA = pd.read_csv('../input/uk-covid19-data/England_Regions_COVID_Dataset.csv', index_col='date',parse_dates=True)
UK_LOCAL_AUTHORITY_DATA = pd.read_csv('../input/uk-covid19-data/UK_Local_Authority_UTLA_COVID_Dataset.csv', index_col='date',parse_dates=True)
POPULATION_DATA = pd.read_csv('../input/uk-covid19-data/NEW_Official_Population_Data_ONS_mid-2019.csv',index_col='Unnamed: 0')

UK_AND_NATIONS = pd.concat([UK_TOTAL_DATA,DEVOLVED_NATION_DATA])
ENGLAND_AND_REGIONS = pd.concat([DEVOLVED_NATION_DATA[DEVOLVED_NATION_DATA['areaName'] == 'England'],ENGLAND_REGIONS_DATA])
UK_AND_UTLAS = pd.concat([UK_TOTAL_DATA,UK_LOCAL_AUTHORITY_DATA])

In [ ]:
# Add population density data to UK_LOCAL_AUTHORITY_DATA dataframe
mapping = dict(POPULATION_DATA[['areaCode', 'Population Density (per sq. km)']].values)
UK_LOCAL_AUTHORITY_DATA['Population Density (per sq. km)'] = UK_LOCAL_AUTHORITY_DATA['areaCode'].map(mapping)

# Add population data to UK_LOCAL_AUTHORITY_DATA dataframe
MAPPING_POP = dict(POPULATION_DATA[['areaCode', 'Population']].values)
UK_LOCAL_AUTHORITY_DATA['Population'] = UK_LOCAL_AUTHORITY_DATA['areaCode'].map(MAPPING_POP)

# Add population data to UK_AND_NATIONS dataframe - '.title()' method used to convert country names from all caps so they match with COVID-19 DataFrame
NATION_MAPPING_POP = dict(POPULATION_DATA[['areaName', 'Population']].values)
NATION_MAPPING_POP_fix = dict(zip([i.title() for i in NATION_MAPPING_POP.keys()], [i for i in NATION_MAPPING_POP.values()]))

UK_AND_NATIONS['Population'] = UK_AND_NATIONS['areaName'].map(NATION_MAPPING_POP_fix)

In [ ]:
## visualisation of difference between NATION_MAPPING_POP and NATION_MAPPING_POP_fix for jisoo
print([i for i in NATION_MAPPING_POP][:5])
print([i for i in NATION_MAPPING_POP_fix][:5])

In [ ]:
# Calculate 7-day averages for new deaths and new deaths per 100,000 people

SEVEN_DAY_AVERAGES = pd.Series(dtype='float64')

for i in UK_AND_NATIONS['areaName'].unique():
    SEVEN_DAY_AVERAGES = SEVEN_DAY_AVERAGES.append(UK_AND_NATIONS[UK_AND_NATIONS["areaName"] == i]['newDeaths28DaysByDeathDate'].rolling(window=7).mean())
    
UK_AND_NATIONS['New_Deaths_7_day_average'] = SEVEN_DAY_AVERAGES

# Create new 7 day rolling average deaths per 100,000 people

UK_AND_NATIONS['New_Deaths_7_day_average_per_100,000'] = (UK_AND_NATIONS['New_Deaths_7_day_average']*100000/
                                                                   UK_AND_NATIONS['Population'])

# Calculate 7-day averages for new cases 

AVERAGES_CASES = pd.Series(dtype='float64')

for i in UK_AND_NATIONS['areaName'].unique():
    AVERAGES_CASES = AVERAGES_CASES.append(UK_AND_NATIONS[UK_AND_NATIONS["areaName"] == i]['newCasesByPublishDate'].rolling(window=7).mean())
    
UK_AND_NATIONS['New_Cases_7_day_average'] = AVERAGES_CASES


# Create %population first-dose vaccinated

UK_AND_NATIONS['People_Vaccinated_First_Dose_Percent'] = UK_AND_NATIONS['cumPeopleVaccinatedFirstDoseByPublishDate']*100/UK_AND_NATIONS['Population'].fillna(0)

# UK Total Cumulative Death Rates and Daily New Deaths Over Time
______________________________________________________________________________

In [ ]:
# Plot cumulative death-rate per 100,000 population over time for UK and Devolved Nations
f, ax = plt.subplots(figsize=(15,6))
sns.lineplot(data=UK_AND_NATIONS, x=UK_AND_NATIONS.index, y="cumDeaths28DaysByDeathDateRate", hue="areaName")
plt.title('Cumulative Death-Rate per 100,000 population for UK and Devolved Nations')
ax.grid(linestyle=':', linewidth='0.5')
ax.set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
ax.set_ylim(0)
ax.legend(ncol=2,title=None,loc=(0.45,0.03))


# Full UK Lockdown 1 Text
TEXT = [('2020-03-19','First UK National Lockdown'),
        ('2020-05-24','NHS Test & Trace'),
        ('2020-06-25','First Local Lockdown'),
        ('2020-10-10','3-Tier System'),
        ('2020-10-19','Wales Lockdown'),
        ('2020-11-01','England Lockdown'),
        ('2020-11-28','Pfizer Vaccine Approval'),
        ('2020-12-26','Oxf/AstZen Vaccine Approval')
       ]

for i,j in TEXT[0:6]: 
    ax.text(date.fromisoformat(i), UK_AND_NATIONS['cumDeaths28DaysByDeathDateRate'].max(), j, rotation=90,va='top')
for i,j in TEXT[6:]: 
    ax.text(date.fromisoformat(i), 3, j, rotation=90)

# Vertical Lines

ax.axvline(date.fromisoformat('2020-03-23'), ls='--', color='black')
ax.axvline(date.fromisoformat('2020-05-28'), ls='--', color='black')
ax.axvline(date.fromisoformat('2020-06-29'), ls='--', color='black')
ax.axvline(date.fromisoformat('2020-10-14'), ls='--', color='black')
ax.axvline(date.fromisoformat('2020-10-23'), ls='--', color='black')
ax.axvline(date.fromisoformat('2020-11-05'), ls='--', color='black')
ax.axvline(date.fromisoformat('2020-12-02'), ls='--', color='black')
ax.axvline(date.fromisoformat('2020-12-30'), ls='--', color='black')

plt.savefig('UK_Death_Rates_Overview.jpg',dpi=300)
plt.show



In [ ]:
# Plot Daily New Deaths vs Percent of Population Vaccinated (First-Dose)

NEW_DEATHS_GRAD = np.gradient(UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom']["New_Deaths_7_day_average"].rolling(7).mean())

f, ax1 = plt.subplots(figsize=(15,6))
sns.lineplot(data=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'],
             x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index,
             y="New_Deaths_7_day_average")
plt.title('New Daily Deaths (7-day Rolling Average) and Rate-of-Change for United Kingdom')
ax1.grid(linestyle=':', linewidth='0.5')
ax1.set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
ax1.set_ylim(0)
ax1.fill_between(UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index[-6:-1],0,
                 UK_AND_NATIONS[UK_AND_NATIONS["areaName"]=='United Kingdom']['New_Deaths_7_day_average'][-6:-1],
                 facecolor='tab:blue',alpha=.3)
#ax.legend(ncol=2,title=None,loc=(0.45,0.03))


# Full UK Lockdown 1 Text
TEXT = [('2020-03-19','First UK National Lockdown'),
        ('2020-05-24','NHS Test & Trace'),
        ('2020-06-25','First Local Lockdown'),
        ('2020-10-10','3-Tier System'),
        ('2020-10-19','Wales Lockdown'),
        ('2020-11-01','England Lockdown'),
        ('2020-11-28','Pfizer Vaccine Approval'),
        ('2020-12-26','Oxf/AstZen Vaccine Approval')
       ]

for i,j in TEXT[0:6]: 
    ax1.text(date.fromisoformat(i), UK_AND_NATIONS['New_Deaths_7_day_average'].max()+45, j, rotation=90,va='top')
for i,j in TEXT[6:]: 
    ax1.text(date.fromisoformat(i), 10, j, rotation=90)

# Vertical Lines

ax1.axvline(date.fromisoformat('2020-03-23'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-05-28'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-06-29'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-14'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-23'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-11-05'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-02'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-30'), ls='--', color='black')

# Second axis (Vaccine introduction)

ax2 = ax1.twinx()
color = 'magenta'
ax2.set_ylabel('Rate of Change of New Daily Deaths', color=color)  # we already handled the x-label with ax1
sns.lineplot(x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index,
             y=NEW_DEATHS_GRAD,color=color,linestyle='--',ax=ax2)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(-65,65)
plt.savefig('UK_New_Deaths_and_Rate_of_Change.jpg',dpi=300)
plt.show


In [ ]:
# Plot Daily New Deaths vs Percent of Population Vaccinated (First-Dose)

f, ax1 = plt.subplots(figsize=(15,6))
sns.lineplot(data=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'],
             x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index,
             y="New_Deaths_7_day_average")
plt.title('New Daily Deaths (7-day Rolling Average) and % Population First-Dose Vaccinated for United Kingdom')
ax1.grid(linestyle=':', linewidth='0.5')
ax1.set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
ax1.set_ylim(0)
ax1.fill_between(UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index[-6:-1],0,
                 UK_AND_NATIONS[UK_AND_NATIONS["areaName"]=='United Kingdom']['New_Deaths_7_day_average'][-6:-1],
                 facecolor='tab:blue',alpha=.3)
#ax.legend(ncol=2,title=None,loc=(0.45,0.03))


# Full UK Lockdown 1 Text
TEXT = [('2020-03-19','First UK National Lockdown'),
        ('2020-05-24','NHS Test & Trace'),
        ('2020-06-25','First Local Lockdown'),
        ('2020-10-10','3-Tier System'),
        ('2020-10-19','Wales Lockdown'),
        ('2020-11-01','England Lockdown'),
        ('2020-11-28','Pfizer Vaccine Approval'),
        ('2020-12-26','Oxf/AstZen Vaccine Approval')
       ]

for i,j in TEXT[0:]: 
    ax1.text(date.fromisoformat(i), UK_AND_NATIONS['New_Deaths_7_day_average'].max()+45, j, rotation=90,va='top')
# for i,j in TEXT[6:]: 
#     ax.text(date.fromisoformat(i), 3, j, rotation=90)

# Vertical Lines

ax1.axvline(date.fromisoformat('2020-03-23'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-05-28'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-06-29'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-14'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-23'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-11-05'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-02'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-30'), ls='--', color='black')

# Second axis (Vaccine introduction)

ax2 = ax1.twinx()
color = 'magenta'
ax2.set_ylabel('sin', color=color)  # we already handled the x-label with ax1
sns.lineplot(data=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='England'],
             x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='England'].index,
             y="People_Vaccinated_First_Dose_Percent",color=color,ax=ax2)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0,100)
plt.savefig('UK_Deaths_and_Vaccines delivered.jpg',dpi=300)
plt.show



In [ ]:
# Plot Daily New Deaths vs Percent of Population Vaccinated (First-Dose)

NEW_DEATHS_GRAD = np.gradient(UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom']["New_Cases_7_day_average"].rolling(7).mean())

f, ax1 = plt.subplots(figsize=(15,6))
sns.lineplot(data=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'],
             x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index,
             y="New_Cases_7_day_average")
plt.title('New Daily Confirmed Cases (7-day Rolling Average) and Rate-of-Change for United Kingdom')
ax1.grid(linestyle=':', linewidth='0.5')
ax1.set_xlim(date.fromisoformat('2020-08-01'),UK_AND_NATIONS.index[-2])
ax1.set_ylim(0)
#ax.legend(ncol=2,title=None,loc=(0.45,0.03))


# Full UK Lockdown 1 Text
TEXT = [('2020-03-24','First UK National Lockdown'),
        ('2020-05-27','NHS Test & Trace'),
        ('2020-06-26','First Local Lockdown'),
        ('2020-10-12','3-Tier System'),
        ('2020-10-21','Wales Lockdown'),
        ('2020-11-03','England Lockdown'),
        ('2020-11-30','Pfizer Vaccine Approval'),
        ('2020-12-28','Oxf/AstZen Vaccine Approval')
       ]

for i,j in TEXT[3:]: 
    ax1.text(date.fromisoformat(i), UK_AND_NATIONS['New_Cases_7_day_average'].max()*1.04, j, rotation=90,va='top')
# for i,j in TEXT[6:]: 
#     ax.text(date.fromisoformat(i), 3, j, rotation=90)

# Vertical Lines

# ax1.axvline(date.fromisoformat('2020-03-23'), ls='--', color='black')
# ax1.axvline(date.fromisoformat('2020-05-28'), ls='--', color='black')
# ax1.axvline(date.fromisoformat('2020-06-29'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-14'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-23'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-11-05'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-02'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-30'), ls='--', color='black')

# Second axis (Vaccine introduction)

ax2 = ax1.twinx()
color = 'magenta'
ax2.set_ylabel('Rate of Change of New Daily Cases', color=color)  # we already handled the x-label with ax1
sns.lineplot(x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index,
             y=NEW_DEATHS_GRAD,color=color,linestyle='--',ax=ax2)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(-3500,3500)
plt.savefig('UK_Cases_and_Rate_of_Change.jpg',dpi=300)
plt.show


In [ ]:
LON_DR_NOW = ENGLAND_AND_REGIONS[ENGLAND_AND_REGIONS['areaName']=='London']['cumDeaths28DaysByDeathDateRate'][-2]
NW_DR_NOW = ENGLAND_AND_REGIONS[ENGLAND_AND_REGIONS['areaName']=='North West']['cumDeaths28DaysByDeathDateRate'][-2]
PCT_GAP = round((NW_DR_NOW-LON_DR_NOW)*100/LON_DR_NOW)

In [ ]:
# Plot cumulative deaths per 100,000 population over time for English Regions
f, ax = plt.subplots(figsize=(15,6))
sns.lineplot(data=ENGLAND_AND_REGIONS, x=ENGLAND_AND_REGIONS.index, y="cumDeaths28DaysByDeathDateRate", hue="areaName")
plt.title('Total Cumulative Death-Rate per 100,000 Population for English Regions')
ax.grid(linestyle=':', linewidth='0.5')
ax.set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
ax.set_ylim(0)
ax.legend(ncol=5,title=None)

# Full UK Lockdown 1 Line
ax.axvline(date.fromisoformat('2020-03-23'), ls='--', color='black')
# Full UK Lockdown 1 Text
ax.text(date.fromisoformat('2020-03-19'),
        ENGLAND_AND_REGIONS['cumDeaths28DaysByDeathDateRate'].max()*1.04,
        'First UK National Lockdown',
        rotation=90,va='top')

# London - North-West time-lag - First Wave
ax.annotate("",
            xy=(date.fromisoformat('2020-04-11'),40), xycoords='data',
            xytext=(date.fromisoformat('2020-04-20'),40), textcoords='data',
            arrowprops=dict(arrowstyle="<->",
                            connectionstyle="arc3"),
            )

# London - North-West time-lag - First Wave - annotated
ax.annotate("First wave about\na week ahead\nin London",
            xy=(date.fromisoformat('2020-04-15'),40), xycoords='data',
            xytext=(date.fromisoformat('2020-03-27'),60), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

# North-West - London Cross point
ax.annotate("North-West death-rate\novertakes London on\n5th May 2020",
            xy=(date.fromisoformat('2020-05-13'),65), xycoords='data',
            xytext=(date.fromisoformat('2020-04-13'),95), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

# London - North-West Gap -Current
ax.annotate("",
            xy=(UK_AND_NATIONS.index[-3], LON_DR_NOW), xycoords='data',
            xytext=(UK_AND_NATIONS.index[-3],NW_DR_NOW), textcoords='data',
            arrowprops=dict(arrowstyle="<->",
                            connectionstyle="arc3"),
            )

# London - North-West Gap -Current - annotated
ax.annotate("North-West currently has\n{}% higher cumulative\ndeath-rate than London".format(PCT_GAP),
            xy=(UK_AND_NATIONS.index[-3],LON_DR_NOW+((NW_DR_NOW-LON_DR_NOW)/1.7)), xycoords='data',
            xytext=(date.fromisoformat('2020-09-10'),LON_DR_NOW+((NW_DR_NOW-LON_DR_NOW)/1.4)), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )


plt.savefig('English_regions_death_rates.jpg',dpi=300)
plt.show

**English Regions:**  
For the English regions, the main thing that stands out is the clear **early onset of the virus in London** versus the rest of England. However, despite potentially being protected by the full UK lockdown, which really began on 23rd March([ref](https://fullfact.org/health/coronavirus-lockdown-hancock-claim/)), by roughly 1-2 extra weeks (if using death-rate as the most reliable indicator of prevalence at that time), the **North of England overtook London in May** and then has also been significantly more affected during the second wave. By 3rd Dec, the North-West of England had a **75% higher death rate** per population than London. It's a similar story for the North-East, and to a lesser extent the Midlands. This is all despite significant local lockdown measures throughout the north-west both pre and post the introduction of the 3-tier lockdon system in England.

Whether or not there is evidence here to support the efficacy of the UK's initial lockdown strategy, is difficult to say. It does seem that the peak cumulative death-rates per capita in the first wave seemed to be less outside of London, and it seems likely that the earlier *relative* introduction of lockdown rules played a part in this. However, it also seems that the deaths continued to occur at a higher-rate outside of London for a longer period - particularly in the North of England. There has also been a huge disparity between how different regions have been affected by the recent second wave. In London, even the lower number of deaths that have occurred have tended to be in non-central Boroughs that were less affected first time round, such as Havering. The East Midlands first wave was about 3 weeks behind London, only reaching first wave levels seen for London in November. However, by the start of Decemeber the cumulative death-rate for the whole pandemic has become significantly higher than that of London.

I think it's important to state that just looking at the regional death-rates alone makes it very difficult to ascertain the efficiacy of lockdown policies. There are so many other potential factors involved - I think particularly demographics and underlaying regional health disparities. In future, I will bring in some new data to try to see how these have affected regional death-rates (WIP at bottom of notebook).

# Upper Tier Local Authority (Borough) Analysis
______________________________________________________________________________

**London vs Greater Manchester**

In [ ]:
LB = '''Hackney and City of London
Westminster
Kensington and Chelsea
Hammersmith and Fulham
Wandsworth
Lambeth
Southwark
Tower Hamlets
Islington
Camden
Brent
Ealing
Hounslow
Richmond upon Thames
Kingston upon Thames
Merton
Sutton
Croydon
Bromley
Lewisham
Greenwich
Bexley
Havering
Barking and Dagenham
Redbridge
Newham
Waltham Forest
Haringey
Enfield
Barnet
Harrow
Hillingdon
'''.split("\n")[0:-1]

In [ ]:
# London borough list
LONDON_BOROUGHS = list(set(LB).intersection(UK_LOCAL_AUTHORITY_DATA['areaName'].unique()))
# Greater Manchester borough list
MANC_BOROUGHS = ['Bolton', 'Bury', 'Oldham', 'Rochdale', 'Stockport', 'Tameside',
                 'Trafford', 'Wigan', 'Manchester', 'Salford']
# West Midlands borough list
WM_BOROUGHS = ['Birmingham', 'Coventry', 'Wolverhampton', 'Dudley', 'Sandwell', 'Solihull', 'Walsall']

In [ ]:
f, axs = plt.subplots(1,2,sharex=True, sharey=True,figsize=(17,6))

#f, ax = plt.subplots(figsize=(15,6))
sns.lineplot(data=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(MANC_BOROUGHS)],
             x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(MANC_BOROUGHS)].index,
             y="cumDeaths28DaysByDeathDateRate", hue="areaName",ax=axs[0])
axs[0].set_title('Cumulative Death Rate per 100,000 People for Greater Manchester Boroughs')
axs[0].grid(linestyle=':', linewidth='0.5')
axs[0].set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
axs[0].set_ylim(0)
#axs[0].get_legend().remove()

sns.lineplot(data=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(LONDON_BOROUGHS)],
             x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(LONDON_BOROUGHS)].index,
             y="cumDeaths28DaysByDeathDateRate", hue="areaName",ax=axs[1])
plt.title('Cumulative Death Rate per 100,000 People for London Boroughs')
axs[1].grid(linestyle=':', linewidth='0.5')
axs[1].set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
axs[1].set_ylim(0)
axs[1].get_legend().remove()
plt.savefig('Greater_Manchester_vs_London_Boroughs.jpg',dpi=300)

**Greater Manchester vs London**  
Again this shows the difference in the cumulative death-rates between Greater Manchester and London. You can see that some London boroughs have also shown the second wave increases in deaths, but only the borough of Havering in Essex has shown anything comparable to that seen all across Greater Manchester. Central London boroughs that were some of the worst hit areas in the UK during the first wave like Brent and Harrow have seen much lower numbers of deaths during the second wave.

Interestingly, the lowest death-rate in Greater Manchester is the City of Manchester region itself. Rather than being influenced by differential Lockdown policy, this is likely to be driven by a younger average population compared with the rest of the county, and is something I would like to have a look at.

**Calculate 7-day averages for new deaths**

In [ ]:
SEVEN_DAY_AVERAGES = pd.Series(dtype='float64')

for i in UK_LOCAL_AUTHORITY_DATA['areaName'].unique():
    SEVEN_DAY_AVERAGES = SEVEN_DAY_AVERAGES.append(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"] == i]['newDeaths28DaysByDeathDate'].rolling(window=7).mean())
    
UK_LOCAL_AUTHORITY_DATA['New_Deaths_7_day_average'] = SEVEN_DAY_AVERAGES

In [ ]:
# Create new 7 day rolling average deaths per 100,000 people

UK_LOCAL_AUTHORITY_DATA['New_Deaths_7_day_average_per_100,000'] = (UK_LOCAL_AUTHORITY_DATA['New_Deaths_7_day_average']*100000/
                                                                   UK_LOCAL_AUTHORITY_DATA['Population'])

# Wigan (Greater Manchester) vs Brent (London) vs Havering (Essex)
______________________________________________________________________________

Comparing:  
1) Brent (London - hit early by first wave)  
2) Wigan (North West England - hit significantly by second wave)  
3) Havering (Essex - significantly affected by new 'British' strain of virus affecting South-East England)  

In [ ]:
f, axs = plt.subplots(1,2,sharex=True, sharey=False,figsize=(17,6))

sns.lineplot(data=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(['Brent','Wigan','Havering'])],
             x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(['Brent','Wigan','Havering'])].index,
             y='New_Deaths_7_day_average_per_100,000',
            hue="areaName",ax=axs[0])
ax.grid(linestyle=':', linewidth='0.5')
axs[0].set_title('New Deaths per 100,000 People (7-day Rolling Average)')
axs[0].set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
axs[0].set_ylim(0)
axs[0].grid(linestyle=':', linewidth='0.5')


sns.lineplot(data=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(['Brent','Wigan','Havering'])],
             x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(['Brent','Wigan','Havering'])].index,
             y='cumDeaths28DaysByDeathDateRate',
            hue="areaName",ax=axs[1])
ax.grid(linestyle=':', linewidth='0.5')
axs[1].set_title('Cumulative Death Rate per 100,000 People')
axs[1].grid(linestyle=':', linewidth='0.5')
axs[1].set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
axs[1].set_ylim(0)

plt.savefig('Wigan_vs_Brent_vs_Havering.jpg', dpi=300)
plt.show

In [ ]:
# Date that Brent & Wigan death rates reached 40 per 100,000 people

BRENT_DATE = UK_LOCAL_AUTHORITY_DATA[(UK_LOCAL_AUTHORITY_DATA["areaName"]=='Brent') &
                               (UK_LOCAL_AUTHORITY_DATA["cumDeaths28DaysByDeathDateRate"]<23)].iloc[-1:].index
WIGAN_DATE = UK_LOCAL_AUTHORITY_DATA[(UK_LOCAL_AUTHORITY_DATA["areaName"]=='Wigan') &
                               (UK_LOCAL_AUTHORITY_DATA["cumDeaths28DaysByDeathDateRate"]<23)].iloc[-1:].index

print('Wigan death-rate lag compared to Brent in UK first wave:',BRENT_DATE-WIGAN_DATE)

K = UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(['Brent','Wigan'])]["cumDeaths28DaysByDeathDateRate"].loc['2020-07-01']
D = UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(['Brent','Wigan'])]["cumDeaths28DaysByDeathDateRate"].loc[UK_AND_NATIONS.index[-2]]
BR_WG_PCT = round((K[0]-K[1])*100/K[1])
WG_BR_PCT = round((D[1]-D[0])*100/D[0])

print('On 2020-07-01 Brent had {}% higher death-rate than Wigan'.format(BR_WG_PCT))
print('On {} Wigan has {}% higher death-rate than Brent'.format(UK_AND_NATIONS.index[-2].date(),WG_BR_PCT))

**Wigan vs Brent vs Havering:**  
Looking at just 2 boroughs out of the whole UK can be a bit misleading, however it does highlight the difference in behaviour over the full period of the pandemic between the early hit central London boroughs and the rest of the country.

Looking at the time-lag that it took Wigan to reach the same cumulative death-rate as Brent, Wigan was roughly two weeks behind Brent during the first wave period. By the 1st July (after end of first wave) Brent had a 24.0% higher death-rate than Wigan. This and and the more general regional differences led to the belief that inner-city metropolitan areas were more vulenerable to COVID-19, due to population density (Brent is about 4 times more densely populated than Wigan, although they have similar total populations) and that the first national lockdown had helped stop the spread of the virus into outlying towns and cities. However, by 4th December (time of writing) Wigan had seen its cumulative death-rate rise dramatically, with 62% more deaths per capita than Brent.  

Even more interesting is then to look at Havering, Essex which seemed to affected earlier than Wigan during the first wave, but to a lesser extent. During the second wave it was then affected later than Wigan but significantly more. This is likely due to it's proximity to the presence of the 'British-variant' in the south-east of England, during which time the number of deaths per day in Wigan was actually falling. A small insight into how the complex interaction between previous regional exposure to the virus and the development of new variants  is in the spread and effects of the virus.

# March 30th vs. Latest Data - All UK Boroughs
______________________________________________________________________________

After seeing this difference in behaviour between regions during the first and second major waves in th UK, I've decided to compare the death-rates across UK boroughs on March 30th (one week after Boris Johnson's 'stay-at-home' order that marked the start of the first full national UK lockdown) with the latest data.

The idea here is to compare the eventual outcomes for the areas first most severly hit compared with those that seem to have turned out worse in the long-run. I am also aiming to see if there is a general relationship across the country between how widespread the virus was in a particular area (using death-rate as the most reliable proxy) 1-week after the start of the first lockdown and what the eventual death-rate is turning out to be. The theory being that if the virus was less widespread in an area when the lockdown came into force, it should have led to an overall better outcome for that area relative to places that had already seen high death-rates.

Of course, there are many other factors that could have affected differential death-rates across the regions but if the effect-size of the relationship is strong enough then it potentially could be seen in a regression analysis of the 180 boroughs of the UK.

In [ ]:
# Create dataframe comparing death rates in UK boroughs on March 30th and Nov 27th 2020

MARCH_NOV = pd.DataFrame()
MARCH_NOV['areaName'] = [i for i in UK_LOCAL_AUTHORITY_DATA.loc['2020-03-30']['areaName']]
MARCH_NOV['areaCode'] = [i for i in UK_LOCAL_AUTHORITY_DATA.loc['2020-03-30']['areaCode']]
MARCH_NOV['MARCH_30'] = [i for i in UK_LOCAL_AUTHORITY_DATA.loc['2020-03-30']['cumDeaths28DaysByDeathDateRate']]
MARCH_NOV['MARCH_30'] = MARCH_NOV['MARCH_30'].fillna(0)
MARCH_NOV['Latest_Data'] = [i for i in UK_LOCAL_AUTHORITY_DATA.loc[UK_AND_NATIONS.index[-2]]['cumDeaths28DaysByDeathDateRate']]
MARCH_NOV['Region'] = ['London' if i in LONDON_BOROUGHS else ('Greater Manchester' if i in MANC_BOROUGHS else ('West Midlands' if i in WM_BOROUGHS else 'Rest of UK')) for i in UK_LOCAL_AUTHORITY_DATA.loc['2020-03-30']['areaName']]
MARCH_NOV['Population Density (per sq. km)'] = MARCH_NOV['areaCode'].map(mapping)

In [ ]:
# Linear regressions of log(cumulative death-rates) for March 30th & Latest Data

slope, intercept, r, p, se = scipy.stats.linregress(MARCH_NOV[MARCH_NOV['Latest_Data'].notnull()]['MARCH_30'],
                                                  MARCH_NOV[MARCH_NOV['Latest_Data'].notnull()]['Latest_Data'])



print('Linear Regressions for relationship between cumulative death rate per 100,000 population on 30-Mar-20 and {}:\n'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")))
print('All UK Boroughs:\n',scipy.stats.linregress(MARCH_NOV[MARCH_NOV['Latest_Data'].notnull()]['MARCH_30'],
                                                  MARCH_NOV[MARCH_NOV['Latest_Data'].notnull()]['Latest_Data']))

REG_RESULT = 'Linear Regression:\ny = {}x + {}\nR\u00b2 = {}\np = {}'.format(round(slope,2),round(intercept,2),round(r**2,2),round(p,3))

In [ ]:
f, ax = plt.subplots(figsize=(15,6))

sns.scatterplot(data=MARCH_NOV,
                x='MARCH_30',y='Latest_Data',hue="Region",style="Region")
plt.title('Relationship between Cumulative Death Rates per 100,000 People for UK Boroughs on 30-Mar-20 and {}'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")))
# ax.set_xlim(0)
# ax.set_ylim(0)
ax.set_ylabel('Total Cumulative Death-Rates per 100,000 on {}'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")))
ax.set_xlabel('Total Cumulative Death-Rates per 100,000 on 30-Mar-20')
ax.annotate('Wigan',
            xy=(MARCH_NOV[MARCH_NOV['areaName']=='Wigan']['MARCH_30'],
                         MARCH_NOV[MARCH_NOV['areaName']=='Wigan']['Latest_Data']),
            xytext=(MARCH_NOV[MARCH_NOV['areaName']=='Wigan']['MARCH_30']+0.1,
                   MARCH_NOV[MARCH_NOV['areaName']=='Wigan']['Latest_Data']+2))
ax.annotate('Brent',
            xy=(MARCH_NOV[MARCH_NOV['areaName']=='Brent']['MARCH_30'],
                         MARCH_NOV[MARCH_NOV['areaName']=='Wigan']['Latest_Data']),
            xytext=(MARCH_NOV[MARCH_NOV['areaName']=='Brent']['MARCH_30']+0.1,
                   MARCH_NOV[MARCH_NOV['areaName']=='Brent']['Latest_Data']+2))

ax.annotate('Havering',
            xy=(MARCH_NOV[MARCH_NOV['areaName']=='Havering']['MARCH_30'],
                         MARCH_NOV[MARCH_NOV['areaName']=='Wigan']['Latest_Data']),
            xytext=(MARCH_NOV[MARCH_NOV['areaName']=='Havering']['MARCH_30']+0.1,
                   MARCH_NOV[MARCH_NOV['areaName']=='Havering']['Latest_Data']+2))

ax.annotate(REG_RESULT,
            xy=(MARCH_NOV[MARCH_NOV['areaName']=='Brent']['MARCH_30'],
                         MARCH_NOV[MARCH_NOV['areaName']=='Wigan']['Latest_Data']),
            xytext=(25,25))

plt.plot(range(int(math.ceil(MARCH_NOV['MARCH_30'].max()))), slope*range(int(math.ceil(MARCH_NOV['MARCH_30'].max()))) + intercept,linestyle='--')

plt.savefig('March_30_vs_Latest_data_all_UK_boroughs.jpg',dpi=300)
plt.show

**Analysis written on 4th Dec 2020:**  
As can be seen on the scatter plot above, there is no clear, statisitically significant relationship between the cumulative death-rates observed across UK boroughs between March 30th 2020 and the latest available data (4th Dec 2020). This is confirmed with the R-values and p-values for the Pearson linear regression analyses above. (Skewness and Kurtosis testing below, output hidden). It doesn't matter which date at the start of the crisis is chosen, the same relationship is observed - March 30th was chosen as it was early enough to show the differential spread of the virus, but late enough that all but a handful of boroughs had not seen any deaths (one of which was Eric Idle's favourite place of Rutland for Python aficionados).

I've highlighted the locations of Wigan and Brent on the plot as it further highlights the differences between the beginning of the crisis and now. It seems as though the population density and international nature of central London boroughs has given way to the older, less healthy populations in the the North of England. However, this is only a theory and needs more data and analysis to confirm. This is something I am planning to scrape from the ONS website in future.



In [ ]:
from scipy import stats

# Distplot of cumulative death rates just to check pearson regression is appropriate - close enough. Taking sqrt was better for kurtosis and skew but the regressions came out very similar anyway as there is no significant relationship
f, axs = plt.subplots(1,3,figsize=(18,4))
sns.distplot(MARCH_NOV['Latest_Data'],ax=axs[0])
sns.distplot(np.log(MARCH_NOV['Latest_Data']),ax=axs[1])
sns.distplot((MARCH_NOV['Latest_Data']**0.5), ax=axs[2])

f, axs = plt.subplots(1,3,figsize=(18,4))
sns.distplot(MARCH_NOV['MARCH_30'],ax=axs[0])
sns.distplot(np.log(MARCH_NOV['MARCH_30']).replace([np.inf, -np.inf], np.nan), ax=axs[1])
sns.distplot((MARCH_NOV['MARCH_30']**0.5), ax=axs[2])

In [ ]:
from scipy.stats import kurtosis,kurtosistest, skew
print('Latest Data:')
print('Kurtosis:',kurtosis((MARCH_NOV['Latest_Data']),nan_policy='omit'))
#print(kurtosis(np.log(MARCH_NOV['Latest_Data']),nan_policy='omit'))
#print(kurtosis((MARCH_NOV['Latest_Data']**0.5),nan_policy='omit'))

print('Skew:',skew((MARCH_NOV['Latest_Data']),nan_policy='omit'))
#print(skew(np.log(MARCH_NOV['Latest_Data']),nan_policy='omit'))
#print(skew((MARCH_NOV['Latest_Data']**0.5),nan_policy='omit'))

print('March 30th:')
print('Kurtosis:',kurtosis((MARCH_NOV['MARCH_30']),nan_policy='omit'))
#print(kurtosis(np.log(MARCH_NOV['MARCH_30']),nan_policy='omit'))
#print(kurtosis((MARCH_NOV['MARCH_30']**0.5),nan_policy='omit'))

print('Skew:',skew((MARCH_NOV['MARCH_30']),nan_policy='omit'))
#print(skew(np.log(MARCH_NOV['MARCH_30']),nan_policy='omit'))
#print(skew((MARCH_NOV['MARCH_30']**0.5),nan_policy='omit'))

**Note:** Skewness for Death-Rates on March 30th 2020 on high side for a linear regression but as the analysis is showing that there is no significant relationship between total deaths rates on March 30th and now then it doesn't matter too much.

# Top 10 Worst Boroughs: March 30th 2020 vs Date of Latest Data

Plan here is to do some more qualitiative visualisations to help understand the difference across regions between the start of the crisis and now

In [ ]:
# top 10 worst utlas on March 30th and latest available data at the time of notebook commit

WORST_UTLAS_MAR = MARCH_NOV[['areaName','MARCH_30','Latest_Data']].sort_values(by=['MARCH_30'],ascending=False, na_position='last').iloc[0:10]
print('Top 10 Worst UK Boroughs for cumulative Death Rate per 100k pop on Mar-30:\n',WORST_UTLAS_MAR)

WORST_UTLAS_NOV = MARCH_NOV[['areaName','MARCH_30','Latest_Data']].sort_values(by=['Latest_Data'],ascending=False, na_position='last').iloc[0:10].sort_values(by=['MARCH_30'],ascending=False)
print('\nTop 10 Worst UK Boroughs for cumulative Death Rate per 100k pop on {}:\n'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")),WORST_UTLAS_NOV)



In [ ]:
import matplotlib.patches as patches

f, axs = plt.subplots(1,2,sharex=True, sharey=True,figsize=(17,6))

sns.lineplot(data=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(WORST_UTLAS_MAR['areaName'])],
             x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(WORST_UTLAS_MAR['areaName'])].index,
             y='New_Deaths_7_day_average_per_100,000',
             hue="areaName",ax=axs[0])
axs[0].grid(linestyle=':', linewidth='0.5')
axs[0].set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
axs[0].set_ylim(0)
axs[0].set_title('Top Ten Worst Boroughs for Cumulative Death-Rate on 30-Mar-20')

sns.lineplot(data=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(WORST_UTLAS_NOV['areaName'])],
             x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(WORST_UTLAS_NOV['areaName'])].index,
             y='New_Deaths_7_day_average_per_100,000',
             hue="areaName",ax=axs[1])
axs[1].grid(linestyle=':', linewidth='0.5')
axs[1].grid(linestyle=':', linewidth='0.5')
axs[1].set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
axs[1].set_ylim(0)
axs[1].set_title('Top Ten Worst Boroughs for Cumulative Death-Rate on {}'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")))
plt.savefig('Top ten worst boroughs march30th vs latest data')


**Top 10 Worst Boroughs on March 30th and Latest Data:**  
These lineplots show what I think are the three key differences between the top ten worst UK boroughs at the start of the crisis and the top ten worst boroughs currently (writing on 4th Dec).  

1) **March** - The earlier onset of the increase in deaths in the worst-affect first-wave areas by roughly 1-2 weeks  

2) **May & June** - A more rapid decline in daily deaths in the areas first affected, as opposed to a slower decling in the current worst boroughs  

3) **October-December** - Much more signifcant daily death numbers in the current worst peforming boroughs during the second wave. In fact, in these boroughs the second wave has been comparable or even worse than the first wave.  

Again, there is no evidence here to rule out any benefits of lockdown policies in reducing total cumulative deaths, but it does show that the initial lockdown period beginning in March 2020 was not successful in preventing the virus becoming very prevalent across all boroughs in the UK, even those that were relatively less affected early on.

This is shown in a more visual way in the ridge plot below:

In [ ]:
from matplotlib.dates import DateFormatter

f, axs = plt.subplots(20,1,sharex=True, sharey=True,figsize=(15,8))

for i,j in zip(range(20),[k for k in WORST_UTLAS_NOV["areaName"]][::-1]+[p for p in WORST_UTLAS_MAR["areaName"]][::-1]):
    sns.lineplot(x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j].index,
                 y=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'],
                 color="w",
                 ax=axs[i])
    axs[i].fill_between(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j].index,
                        0, UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'],
                        facecolor='blue',alpha=.4)
    axs[i].get_yaxis().set_visible(False)
    
    axs[i].set_frame_on(False)
    axs[i].text(UK_AND_NATIONS.index[-1], 0.1, j, rotation=0)
    axs[i].xaxis.set_major_formatter(DateFormatter("%b"))
    axs[i].set_xlabel('')
    axs[i].scatter(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'][0:200].idxmax(),
                   UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'][0:200].max(),
                   color='r')
    axs[i].set_xlim(date.fromisoformat('2020-03-10'))
    
axs[9].axhline(0, ls='--', color='b',alpha=.4)    
axs[0].set_title('Ridge Plot for 10 Worst UK Boroughs on 30-Mar-20 and {}\nDaily Deaths per 100,000 People (7-day Average)'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")),
                fontsize = 'xx-large', pad=-30)

axs[8].text(date.fromisoformat('2020-03-10'), 0, 'Worst Boroughs on {}'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")), rotation=90,fontsize='medium')
axs[18].text(date.fromisoformat('2020-03-10'), 0, 'Worst Boroughs on 30-Mar-20', rotation=90,fontsize='medium')

f.subplots_adjust(hspace=-.85)
plt.savefig('Ridge_Plot_First_Wave_vs_Second_Wave.jpg',dpi=300)

**Worst UK Boroughs: First Wave vs. Second Wave**  

The ridge plot above shows the 7-day average daily deaths for the inital worst boroughs versus those with the highest deaths rates now (4th Dec). The red dots represent the date of the peak 7-day average daily death numbers in the first wave.

Again you can clearly see the 3 key differences in behaviour:  
**First Wave Worst Boroughs:** Earlier onset, higher first wave peak, rapid drop-off, no second wave  
**Latest Data Worst Boroughs:** Later onset, lower first peak, more gradual drop-off, very severe second wave  

# Ridge Plot and Heat Map for all UK Boroughs (WIP)
______________________________________________________________________________

In [ ]:
# Create list UK boroughs ordered by date of peak new deaths (7-day average)

f = []

for i in UK_LOCAL_AUTHORITY_DATA['areaName'].unique():
    f.append(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA['areaName']==i]['New_Deaths_7_day_average_per_100,000'].fillna(0).iloc[:210].idxmax())

p = sorted(zip(f,UK_LOCAL_AUTHORITY_DATA['areaName'].unique()))

UTLA_BY_FIRST_PEAK = [j for i,j in p][::-1]

In [ ]:
# Mountain range for all UK boroughs ordered by date of peak new deaths (7-day average) during first wave

f, axs = plt.subplots(len(UTLA_BY_FIRST_PEAK),1,sharex=True, sharey=True,figsize=(15,10))

for i,j in zip(range(len(UTLA_BY_FIRST_PEAK)),UTLA_BY_FIRST_PEAK):
    sns.lineplot(x=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j].index,
                 y=UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'],
                 color="w",
                 ax=axs[i])
    axs[i].fill_between(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j].index,
                        0, UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'],
                        facecolor='blue',alpha=.4)
    axs[i].set_xlim(date.fromisoformat('2020-03-01'),UK_AND_NATIONS.index[-2])
    axs[i].set_ylim(0,6)
    axs[i].set_axis_off()
    axs[i].scatter(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'].fillna(0)[0:200].idxmax(),
                   UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"]==j]['New_Deaths_7_day_average_per_100,000'].fillna(0)[0:200].max(),
                   color='r')

axs[0].set_title('Ridge Plot for all UK Boroughs on {} ordered by date of first-wave peak\n(Daily Deaths per 100,000 People (7-day Average))'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")),
                fontsize = 'xx-large', pad=-20)

f.subplots_adjust(hspace=-.95)
plt.savefig('Ridge_plot_all_UK_boroughs.jpg',dpi=300)

In [ ]:
WORST_BOROUGHS = MARCH_NOV[MARCH_NOV['Latest_Data'].notnull()][['areaName','MARCH_30','Latest_Data']].sort_values(by=['Latest_Data'],ascending=False, na_position='last')['areaName'].unique()

In [ ]:
# Create list UK boroughs ordered by date of peak new deaths (7-day average)

f = []

for i in UK_LOCAL_AUTHORITY_DATA['areaName'].unique():
    if UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA['areaName']==i]['cumDeaths28DaysByDeathDateRate'][:-2][UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA['areaName']==i]['cumDeaths28DaysByDeathDateRate'][:-2].fillna(0).ge(2)].any():
        f.append(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA['areaName']==i]['cumDeaths28DaysByDeathDateRate'][:-2][UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA['areaName']==i]['cumDeaths28DaysByDeathDateRate'][:-2].fillna(0).ge(2)].index[0])

p = sorted(zip(f,UK_LOCAL_AUTHORITY_DATA['areaName'].unique()))

UTLA_BY_DATE_AT_2 = [j for i,j in p][::-1]


In [ ]:
# Create dataframe for heat map
HEATMAP_DATA = UK_LOCAL_AUTHORITY_DATA[['areaName','New_Deaths_7_day_average_per_100,000']].fillna(0)

HMD_2 = pd.DataFrame(index=HEATMAP_DATA[HEATMAP_DATA["areaName"]=='Aberdeen City'].index)

for i in UTLA_BY_FIRST_PEAK:

    HMD_2 = HMD_2.join(HEATMAP_DATA[HEATMAP_DATA["areaName"]==i]['New_Deaths_7_day_average_per_100,000'],rsuffix=i)
    
HMD_2 = HMD_2.rename(columns=dict(zip(HMD_2.columns,UTLA_BY_FIRST_PEAK)))

HMD_2.index = HMD_2.index.strftime('%d-%b-%y')

In [ ]:
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

f2, ax2 = plt.subplots(figsize=(15,8))
sns.heatmap(data=HMD_2.iloc[70:].T,cmap="rainbow",xticklabels=30,vmax=3,ax=ax2)
ax2.get_yaxis().set_visible(False)
ax2.set_title('Heatmap for all UK Boroughs on {} ordered by date of first-wave peak\n(Daily Deaths per 100,000 People (7-day Average))'.format(UK_AND_NATIONS.index[-2].strftime("%d-%b-%y")))
plt.show